In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [ ]:
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [3]:
cleansers_url = []
for i in range (1, 4):
    url = f'https://www.stylevana.com/en_EU/skincare/facial-cleanser/face-wash-cleansers.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

36


In [4]:
for i in range (4, 20):
    url = f'https://www.stylevana.com/en_EU/skincare/facial-cleanser/face-wash-cleansers.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

228


In [6]:
for i in range (20, 41):
    url = f'https://www.stylevana.com/en_EU/skincare/facial-cleanser/face-wash-cleansers.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

480


In [7]:
for i in range (41, 87):
    url = f'https://www.stylevana.com/en_EU/skincare/facial-cleanser/face-wash-cleansers.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

1032


In [ ]:
with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [30]:
cleansers_url[1031]

'https://www.stylevana.com/en_EU/enough-project-enuf-proj-cleansing-oil-150ml.html'

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [19]:
#test_url = 'https://www.stylevana.com/en_EU/laneige-lip-sleeping-mask.html'
cleansers_url[39]

'https://www.stylevana.com/en_EU/farm-stay-pure-natural-cleansing-water-500ml-green-tea-seed.html'

In [10]:
cleansers = []

In [31]:
for link in cleansers_url[750:1032]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    name = soup.find('h1', class_='product-name-h1').text
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        
    cleansers_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients}
    
    cleansers.append(cleansers_info)

In [32]:
print(len(cleansers))

1032


## 3. PASARLO A DATAFRAME

In [33]:
cleansers_df = pd.DataFrame(cleansers)
cleansers_df

,name,details,ingredients
0,iUNIK - Calendula Complete Cleansing Oil - 200ml,Product Type: Cleansing Oil\n\n\nSkin Concern:...,[]
1,Krave - Matcha Hemp Hydrating Cleanser 120ml,Product Type: Cleansing Gel\n\n\nSkin Concern:...,[]
2,COSRX - Low pH Good Morning Gel Cleanser - 150ml,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Water, Cocamidopropyl Betaine, Sodium Lauroyl ..."
3,heimish - All Clean Balm,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Ethylhexyl Palmitate, Cetyl Ethylhexanoate, PE..."
4,SOME BY MI - AHA-BHA-PHA 30days Miracle Acne C...,Product Type: Cleansing Water\n\n\nSkin Concer...,[]
...,...,...,...
1027,Etude House - SoonJung 5.5 Foam Cleanser - 100ml,Product Type: Cleansing Foam\n\n\nProduct Unit...,[]
1028,Mandom - Bifesta Foaming Whip Oil Clear - 180g,Product Type: Cleansing Foam\n\n\nProduct Unit...,[]
1029,Fancl - Washing Powder - 50g,Product Type: Cleansing Powder\n\n\nSkin Conce...,"Sodium Cocoyl/Palmoyl/Sunfloweroyl Glutamate,S..."
1030,DeARANCHY - Purifying Pure Cleansing Water - 3...,Product Type: Cleansing Water\n\n\nSkin Concer...,[]


In [34]:
cleansers = cleansers_df.to_csv('data/cleansers.csv')